# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto #

#### Author : Ratnesh Mehrotra #####

### Part 3: Clustering and Segmentation of Toronto Neighbourhoods ###

In [116]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [117]:
# Read the Wikipedia page
page = requests.get(" https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [118]:
# Use of Beautiful Soup HTML parser
soup = BeautifulSoup(page.content, 'html.parser')
table_body = soup.find_all('table', class_='wikitable sortable')
rows = table_body[0].find_all('tr')

all_rows = []
for row in rows:
    cols=row.find_all('td')
    if (len(cols) != 0):
        cols=[x.text.strip() for x in cols]
        all_rows.append(cols)

df=pd.DataFrame(all_rows,columns=['Postcode','Borough','Neighbourhood'])
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [119]:
# Get names of indexes for which Borough == "Not assigned", and remove those rows
indexNames = df[ df['Borough'] == 'Not assigned' ].index
 
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df = df.reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [120]:
# replace "Not assigned" for Neighbourhood with the Borough name
col_names =  ['Postcode', 'Borough', 'Neighbourhood']
df_new = pd.DataFrame(columns = col_names)
for index, row in df.iterrows():
    if row[2] == "Not assigned":
        #df_new = df_new.append(row)
        df_new.loc[index] = [row[0], row[1], row[1]]
    else:
        df_new.loc[index] = [row[0], row[1], row[2]]


df_new = df_new.reset_index(drop=True)
df_new.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [121]:
# concatenate neighbourhood belonging to same postal code 
# Iterate over rows in dataframe, and build a dictionary to check for Neighbourhood with same postal code
# Key in dictionary is Postal Code. Values are tuples with PostalCode, Borough, and Neighbour or string of neighbourhood

col_names =  ['Postcode', 'Borough', 'Neighbourhood']
temp_dict = {}

for index, row in df.iterrows():
    if row[0] in temp_dict:
        temp_dict[row[0]] = ( row[0], row[1], temp_dict.get(row[0],"")[2] + " , "+ row[2] )
    else: 
        temp_dict[row[0]] = ( row[0], row[1], row[2] )

temp_list = list(temp_dict.values())
df_groupedby = pd.DataFrame.from_records(temp_list, columns=col_names)
df_groupedby.head(10)

,Postcode,Borough,Neighbourhood
0,M9P,Etobicoke,Westmount
1,M5P,Central Toronto,"Forest Hill North , Forest Hill West"
2,M6E,York,Caledonia-Fairbanks
3,M3N,North York,Downsview Northwest
4,M4P,Central Toronto,Davisville North
5,M1E,Scarborough,"Guildwood , Morningside , West Hill"
6,M4B,East York,"Woodbine Gardens , Parkview Hill"
7,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
8,M6K,West Toronto,"Brockton , Exhibition Place , Parkdale Village"
9,M4M,East Toronto,Studio District


#### Printing the number of rows in final dataframe. ####

In [122]:
df_groupedby.shape

(103, 3)

### Section 2: Adding latitude & Longitude information ###
##### Note: Using the provided CSV file as Google service failed to work hence using the CSV  #####

In [123]:
# read the lat long CSV file
df_latlong = pd.read_csv("http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")
df_latlong = df_latlong.rename(columns={'Postal Code': 'Postcode'})
df_groupedby_latlong = pd.merge(df_groupedby, df_latlong,on='Postcode')

In [124]:
df_groupedby_latlong.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9P,Etobicoke,Westmount,43.696319,-79.532242
1,M5P,Central Toronto,"Forest Hill North , Forest Hill West",43.696948,-79.411307
2,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
3,M3N,North York,Downsview Northwest,43.761631,-79.520999
4,M4P,Central Toronto,Davisville North,43.712751,-79.390197
5,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
6,M4B,East York,"Woodbine Gardens , Parkview Hill",43.706397,-79.309937
7,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
8,M6K,West Toronto,"Brockton , Exhibition Place , Parkdale Village",43.636847,-79.428191
9,M4M,East Toronto,Studio District,43.659526,-79.340923


### Section 3:  Slice the original dataframe and create a new dataframe of the  Downtown Toronto ###

In [125]:
toronto_data = df_groupedby_latlong[df_groupedby_latlong['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4X,Downtown Toronto,"Cabbagetown , St. James Town",43.667967,-79.367675
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
2,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
3,M5K,Downtown Toronto,"Design Exchange , Toronto Dominion Centre",43.647177,-79.381576
4,M5V,Downtown Toronto,"CN Tower , Bathurst Quay , Island airport , Ha...",43.628947,-79.394420
5,M5L,Downtown Toronto,"Commerce Court , Victoria Hotel",43.648198,-79.379817
6,M5T,Downtown Toronto,"Chinatown , Grange Park , Kensington Market",43.653206,-79.400049
7,M5A,Downtown Toronto,"Harbourfront , Regent Park",43.654260,-79.360636
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564
9,M5B,Downtown Toronto,"Ryerson , Garden District",43.657162,-79.378937


#### Geographical coordinates of Downtown Toronto. ####

In [126]:
address = 'Toronto'

geolocator = Nominatim(user_agent='any_arbit_string')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto's neighborhoods ####

In [127]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Section 4:  Explore Neighborhoods in Toronto ###

#### Define Foursquare Credentials and Version ####

In [128]:
# VERSION = '20180605' # Foursquare API version
CLIENT_ID = 'Y4VM2TNMLHIPH52JYEE4WKPKX5V00WATDYQKNZ42TNP1X4YR' # your Foursquare ID
CLIENT_SECRET = 'E1YFCEWVHSTHXQL0O12LVXCLWO5VSYD1NWYT1OKNXP3VVQXS' # your Foursquare Secret
VERSION = '20180604'

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius


In [129]:
#Get the neighborhood's name.
toronto_data.loc[0, 'Neighbourhood']
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Cabbagetown , St. James Town are 43.667967, -79.3676753.


In [130]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=Y4VM2TNMLHIPH52JYEE4WKPKX5V00WATDYQKNZ42TNP1X4YR&client_secret=E1YFCEWVHSTHXQL0O12LVXCLWO5VSYD1NWYT1OKNXP3VVQXS&v=20180604&ll=43.667967,-79.3676753&radius=500&limit=100'

In [131]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c3766e8351e3d26291ca4c0'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b646a6ff964a5205cb12ae3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/diner_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d147941735',
         'name': 'Diner',
         'pluralName': 'Diners',
         'primary': True,
         'shortName': 'Diner'}],
       'id': '4b646a6ff964a5205cb12ae3',
       'location': {'address': '601 Parliament St.',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Wellesley St. E',
        'distance': 140,
        'formattedAddress': ['601 Parliament St. (at Wellesley St. E)',
         'Toronto ON M4X 1P9',
         'Canada'],
        'labeledLatLngs': [{

In [132]:
# function of FourSquare that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [133]:
# Now we are ready to clean the json and structure it into a *pandas* dataframe.

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Cranberries,Diner,43.667843,-79.369407
1,Butter Chicken Factory,Indian Restaurant,43.667072,-79.369184
2,F'Amelia,Italian Restaurant,43.667536,-79.368613
3,Kingyo Toronto,Japanese Restaurant,43.665895,-79.368415
4,Murgatroid,Restaurant,43.667381,-79.369311


In [134]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

43 venues were returned by Foursquare.


In [135]:
# function to get venues given the latitude and longitude of neighbourhood...to be called recursively for all neighbourhood of Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
      
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        print("name:"+name)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [136]:
# getting the venues for all neighbourhood
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

name:Cabbagetown , St. James Town
name:Berczy Park
name:Rosedale
name:Design Exchange , Toronto Dominion Centre
name:CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara
name:Commerce Court , Victoria Hotel
name:Chinatown , Grange Park , Kensington Market
name:Harbourfront , Regent Park
name:Christie
name:Ryerson , Garden District
name:Central Bay Street
name:Church and Wellesley
name:Stn A PO Boxes 25 The Esplanade
name:Harbord , University of Toronto
name:First Canadian Place , Underground city
name:St. James Town
name:Harbourfront East , Toronto Islands , Union Station
name:Adelaide , King , Richmond


In [137]:
print(toronto_venues.shape)
toronto_venues.head()

(1276, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Cabbagetown , St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
1,"Cabbagetown , St. James Town",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
2,"Cabbagetown , St. James Town",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
3,"Cabbagetown , St. James Town",43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
4,"Cabbagetown , St. James Town",43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant


In [138]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 203 uniques categories.


## 3. Analyze Each Neighbourhood

In [139]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Cabbagetown , St. James Town",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Cabbagetown , St. James Town",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Cabbagetown , St. James Town",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Cabbagetown , St. James Town",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Cabbagetown , St. James Town",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
toronto_onehot.shape

(1276, 204)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [141]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide , King , Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"CN Tower , Bathurst Quay , Island airport , Ha...",0.000000,0.000000,0.083333,0.083333,0.166667,0.166667,0.166667,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,"Cabbagetown , St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,...,0.00,0.00,0.00,0.012346,0.000000,0.000000,0.012346,0.000000,0.00,0.012346
5,"Chinatown , Grange Park , Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.050000,0.000000,0.040000,0.010000,0.000000,0.00,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,Church and Wellesley,0.011494,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,0.000000,...,0.00,0.00,0.00,0.000000,0.011494,0.011494,0.000000,0.011494,0.00,0.011494
8,"Commerce Court , Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000
9,"Design Exchange , Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.00,0.00,0.01,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000


#### print each neighborhood along with the top 5 most common venues

In [142]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide , King , Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Restaurant  0.05
3  Italian Restaurant  0.04
4                 Pub  0.04


----CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara----
              venue  freq
0    Airport Lounge  0.17
1   Airport Service  0.17
2  Airport Terminal  0.17
3     Boat or Ferry  0.08
4           Airport  0.08


----Cabbagetown , St. James Town----
         venue  freq
0   Restaurant  0.09
1  Coffee Shop  0.09
2          Pub  0.05
3         Café  0.05
4       Bakery  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1                Café  0.06
2  Italian Restaurant  0.05
3               

#### put that into a *pandas* dataframe

In [143]:
#function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [144]:
# new dataframe to display the top 10 venues for each neighborhood.

In [145]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide , King , Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Asian Restaurant,Gym,Bakery,Bar,Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Beer Bar,Pub,Café,Cheese Shop,Bakery,Steakhouse,Seafood Restaurant
2,"CN Tower , Bathurst Quay , Island airport , Ha...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Boat or Ferry,Sculpture Garden,Plane,Yoga Studio
3,"Cabbagetown , St. James Town",Restaurant,Coffee Shop,Bakery,Pub,Italian Restaurant,Café,Pizza Place,Pharmacy,Market,Bank
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Chinese Restaurant,Sandwich Place,Salad Place,Falafel Restaurant,Ice Cream Shop
5,"Chinatown , Grange Park , Kensington Market",Café,Bar,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Vietnamese Restaurant,Coffee Shop,Bakery,Chinese Restaurant,Mexican Restaurant,Gaming Cafe
6,Christie,Grocery Store,Café,Park,Convenience Store,Nightclub,Baby Store,Restaurant,Diner,Italian Restaurant,Coffee Shop
7,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Burger Joint,Restaurant,Mediterranean Restaurant,Café,Men's Store,Pub
8,"Commerce Court , Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Steakhouse,Gastropub,Deli / Bodega,Bakery
9,"Design Exchange , Toronto Dominion Centre",Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gastropub,Gym,Deli / Bodega,Italian Restaurant,Bar


## 4. Cluster Neighborhoods

In [146]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 2, 0, 3, 0, 4, 0, 3, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [147]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4X,Downtown Toronto,"Cabbagetown , St. James Town",43.667967,-79.367675,3,Restaurant,Coffee Shop,Bakery,Pub,Italian Restaurant,Café,Pizza Place,Pharmacy,Market,Bank
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Restaurant,Beer Bar,Pub,Café,Cheese Shop,Bakery,Steakhouse,Seafood Restaurant
2,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Playground,Trail,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
3,M5K,Downtown Toronto,"Design Exchange , Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gastropub,Gym,Deli / Bodega,Italian Restaurant,Bar
4,M5V,Downtown Toronto,"CN Tower , Bathurst Quay , Island airport , Ha...",43.628947,-79.394420,3,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Boat or Ferry,Sculpture Garden,Plane,Yoga Studio


In [148]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters